In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.stats import multivariate_normal as mvn

from LSTM import LSTM

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

# set plot parameters
params = {'legend.fontsize': 18,
          'figure.figsize': (16, 12),
          'lines.linewidth': 4,
          'axes.labelsize': 24,
          'axes.titlesize':24,
          'axes.linewidth':5,
          'xtick.labelsize':20,
          'ytick.labelsize':20}
plt.rcParams.update(params)
plt.style.use('seaborn-colorblind')
plt.rcParams['pdf.fonttype'] = 42

np.random.seed(123)

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-8mam28k_ because the default path (/home/jaron/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# Set parameters of script

In [2]:
# number of times to randomly pull training samples 
n_trials = 10

# import data

In [3]:
df = pd.read_csv("Data/2021_02_19_MultifunctionalDynamicData.csv")

all_experiments = df.Experiments.values
unique_experiments = np.unique(all_experiments)
n_conditions = len(unique_experiments)

df.head()

,Experiments,Time,PC_OD,PJ_OD,BV_OD,BF_OD,BO_OD,BT_OD,BC_OD,BY_OD,...,CG_OD,ER_OD,RI_OD,CC_OD,DL_OD,DF_OD,Butyrate,Acetate,Lactate,Succinate
0,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,0,0.000471,0.0,0.000471,0.000000,0.000000,0.000471,0.000471,0.0,...,0.000471,0.000471,0.000471,0.0,0.0,0.000471,0.000000,0.000000,28.000000,0.000000
1,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,16,0.465116,0.0,0.029207,0.000000,0.000000,0.249717,0.500651,0.0,...,0.024339,0.327601,0.001460,0.0,0.0,0.392830,23.092697,47.849302,18.910852,26.141885
2,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,32,0.104523,0.0,0.027928,0.000000,0.000000,0.220107,0.380210,0.0,...,0.020739,0.293384,0.000830,0.0,0.0,0.280111,23.996267,38.915218,17.977137,26.884748
3,PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF,48,0.124852,0.0,0.012194,0.000000,0.000000,0.268268,0.333970,0.0,...,0.023660,0.188188,0.002366,0.0,0.0,0.224952,24.839219,34.325914,19.406971,31.628061
4,BV-BF-BO-BT-BU-DP-BL-BA-BP-EL-FP-CH-AC-BH-CG-E...,0,0.000000,0.0,0.000388,0.000388,0.000388,0.000388,0.000000,0.0,...,0.000388,0.000388,0.000388,0.0,0.0,0.000000,0.000000,0.000000,28.000000,0.000000


### Define which variables are species and which are metabolites

In [4]:
# specify species and metabolite names 
species = df.columns.values[2:-4]
metabolites =  df.columns.values[-4:]
controls = []
system_variables = np.concatenate((np.array(species), np.array(metabolites)))
system_variables

array(['PC_OD', 'PJ_OD', 'BV_OD', 'BF_OD', 'BO_OD', 'BT_OD', 'BC_OD',
       'BY_OD', 'BU_OD', 'DP_OD', 'BL_OD', 'BA_OD', 'BP_OD', 'CA_OD',
       'EL_OD', 'FP_OD', 'CH_OD', 'AC_OD', 'BH_OD', 'CG_OD', 'ER_OD',
       'RI_OD', 'CC_OD', 'DL_OD', 'DF_OD', 'Butyrate', 'Acetate',
       'Lactate', 'Succinate'], dtype=object)

In [5]:
# for each number of samples to train on 
for n_train in np.append(1, np.arange(5, 70+1)[::5]) :

    # perform k-fold x-validation several times (n_trials) with randomized partitions of data
    for k in range(n_trials):

        # random set of n_train experiments for training 
        train_exps = np.random.choice(unique_experiments, n_train, replace=False)
        train_inds = np.in1d(all_experiments, train_exps)
        
        # pull train and test dataframes
        train_df = df.iloc[train_inds].copy()
        test_df  = df.iloc[~train_inds].copy()

        # train LSTM 
        lstm = LSTM(train_df, system_variables, hidden_size=4098)
        lstm.train(train_df)

        # Make predictions
        df_boot_lstm = lstm.predict(test_df)

        # save kfold predictions
        df_boot_lstm.to_csv(f"Fig2/Bootstrap/LSTM_{n_train}samples_trial_{k+1}.csv", index=False)

Epoch: 1/200, Train Loss: 0.00009
Epoch: 2/200, Train Loss: 0.01169
Epoch: 3/200, Train Loss: 0.01816
Epoch: 4/200, Train Loss: 0.00353
Epoch: 5/200, Train Loss: 0.04825
Epoch: 6/200, Train Loss: 0.04743
Epoch: 7/200, Train Loss: 0.05259
Epoch: 8/200, Train Loss: 0.01901
Epoch: 9/200, Train Loss: 0.00081
Epoch: 10/200, Train Loss: 0.00038
Epoch: 11/200, Train Loss: 0.00056
Epoch: 12/200, Train Loss: 0.00058
Epoch: 13/200, Train Loss: 0.00058
Epoch: 14/200, Train Loss: 0.00060
Epoch: 15/200, Train Loss: 0.00067
Epoch: 16/200, Train Loss: 0.00063
Epoch: 17/200, Train Loss: 0.00063
Epoch: 18/200, Train Loss: 0.00062
Epoch: 19/200, Train Loss: 0.00060
Epoch: 20/200, Train Loss: 0.00062
Epoch: 21/200, Train Loss: 0.00056
Epoch: 22/200, Train Loss: 0.00054
Epoch: 23/200, Train Loss: 0.00051
Epoch: 24/200, Train Loss: 0.00049
Epoch: 25/200, Train Loss: 0.00047
Epoch: 26/200, Train Loss: 0.00047
Epoch: 27/200, Train Loss: 0.00047
Epoch: 28/200, Train Loss: 0.00047
Epoch: 29/200, Train Loss: 0.

Epoch: 33/200, Train Loss: 0.00040
Epoch: 34/200, Train Loss: 0.00039
Epoch: 35/200, Train Loss: 0.00037
Epoch: 36/200, Train Loss: 0.00035
Epoch: 37/200, Train Loss: 0.00035
Epoch: 38/200, Train Loss: 0.00035
Epoch: 39/200, Train Loss: 0.00034
Epoch: 40/200, Train Loss: 0.00034
Epoch: 41/200, Train Loss: 0.00034
Epoch: 42/200, Train Loss: 0.00034
Epoch: 43/200, Train Loss: 0.00033
Epoch: 44/200, Train Loss: 0.00032
Epoch: 45/200, Train Loss: 0.00033
Epoch: 46/200, Train Loss: 0.00032
Epoch: 47/200, Train Loss: 0.00032
Epoch: 48/200, Train Loss: 0.00031
Epoch: 49/200, Train Loss: 0.00032
Epoch: 50/200, Train Loss: 0.00032
Epoch: 51/200, Train Loss: 0.00032
Epoch: 52/200, Train Loss: 0.00031
Epoch: 53/200, Train Loss: 0.00030
Epoch: 54/200, Train Loss: 0.00030
Epoch: 55/200, Train Loss: 0.00031
Epoch: 56/200, Train Loss: 0.00030
Epoch: 57/200, Train Loss: 0.00031
Epoch: 58/200, Train Loss: 0.00030
Epoch: 59/200, Train Loss: 0.00031
Epoch: 60/200, Train Loss: 0.00030
Epoch: 61/200, Train

Epoch: 65/200, Train Loss: 0.00047
Epoch: 66/200, Train Loss: 0.00047
Epoch: 67/200, Train Loss: 0.00044
Epoch: 68/200, Train Loss: 0.00047
Epoch: 69/200, Train Loss: 0.00043
Epoch: 70/200, Train Loss: 0.00047
Epoch: 71/200, Train Loss: 0.00047
Epoch: 72/200, Train Loss: 0.00047
Epoch: 73/200, Train Loss: 0.00043
Epoch: 74/200, Train Loss: 0.00047
Epoch: 75/200, Train Loss: 0.00043
Epoch: 76/200, Train Loss: 0.00047
Epoch: 77/200, Train Loss: 0.00043
Epoch: 78/200, Train Loss: 0.00047
Epoch: 79/200, Train Loss: 0.00043
Epoch: 80/200, Train Loss: 0.00047
Epoch: 81/200, Train Loss: 0.00047
Epoch: 82/200, Train Loss: 0.00047
Epoch: 83/200, Train Loss: 0.00047
Epoch: 84/200, Train Loss: 0.00047
Epoch: 85/200, Train Loss: 0.00043
Epoch: 86/200, Train Loss: 0.00043
Epoch: 87/200, Train Loss: 0.00043
Epoch: 88/200, Train Loss: 0.00043
Epoch: 89/200, Train Loss: 0.00046
Epoch: 90/200, Train Loss: 0.00046
Epoch: 91/200, Train Loss: 0.00046
Epoch: 92/200, Train Loss: 0.00046
Epoch: 93/200, Train

Epoch: 97/200, Train Loss: 0.00009
Epoch: 98/200, Train Loss: 0.00008
Epoch: 99/200, Train Loss: 0.00009
Epoch: 100/200, Train Loss: 0.00008
Epoch: 101/200, Train Loss: 0.00009
Epoch: 102/200, Train Loss: 0.00008
Epoch: 103/200, Train Loss: 0.00008
Epoch: 104/200, Train Loss: 0.00009
Epoch: 105/200, Train Loss: 0.00008
Epoch: 106/200, Train Loss: 0.00009
Epoch: 107/200, Train Loss: 0.00008
Epoch: 108/200, Train Loss: 0.00008
Epoch: 109/200, Train Loss: 0.00008
Epoch: 110/200, Train Loss: 0.00008
Epoch: 111/200, Train Loss: 0.00009
Epoch: 112/200, Train Loss: 0.00008
Epoch: 113/200, Train Loss: 0.00008
Epoch: 114/200, Train Loss: 0.00008
Epoch: 115/200, Train Loss: 0.00008
Epoch: 116/200, Train Loss: 0.00009
Epoch: 117/200, Train Loss: 0.00008
Epoch: 118/200, Train Loss: 0.00009
Epoch: 119/200, Train Loss: 0.00008
Epoch: 120/200, Train Loss: 0.00009
Epoch: 121/200, Train Loss: 0.00008
Epoch: 122/200, Train Loss: 0.00008
Epoch: 123/200, Train Loss: 0.00009
Epoch: 124/200, Train Loss: 0.0

Epoch: 129/200, Train Loss: 0.00018
Epoch: 130/200, Train Loss: 0.00018
Epoch: 131/200, Train Loss: 0.00018
Epoch: 132/200, Train Loss: 0.00018
Epoch: 133/200, Train Loss: 0.00015
Epoch: 134/200, Train Loss: 0.00015
Epoch: 135/200, Train Loss: 0.00018
Epoch: 136/200, Train Loss: 0.00015
Epoch: 137/200, Train Loss: 0.00015
Epoch: 138/200, Train Loss: 0.00015
Epoch: 139/200, Train Loss: 0.00018
Epoch: 140/200, Train Loss: 0.00018
Epoch: 141/200, Train Loss: 0.00015
Epoch: 142/200, Train Loss: 0.00018
Epoch: 143/200, Train Loss: 0.00018
Epoch: 144/200, Train Loss: 0.00015
Epoch: 145/200, Train Loss: 0.00018
Epoch: 146/200, Train Loss: 0.00018
Epoch: 147/200, Train Loss: 0.00015
Epoch: 148/200, Train Loss: 0.00018
Epoch: 149/200, Train Loss: 0.00018
Epoch: 150/200, Train Loss: 0.00018
Epoch: 151/200, Train Loss: 0.00018
Epoch: 152/200, Train Loss: 0.00015
Epoch: 153/200, Train Loss: 0.00015
Epoch: 154/200, Train Loss: 0.00015
Epoch: 155/200, Train Loss: 0.00018
Epoch: 156/200, Train Loss: 

...

Epoch: 157/200, Train Loss: 0.06713
Epoch: 158/200, Train Loss: 0.06828
Epoch: 159/200, Train Loss: 0.06852
Epoch: 160/200, Train Loss: 0.06700
Epoch: 161/200, Train Loss: 0.06914
Epoch: 162/200, Train Loss: 0.06831
Epoch: 163/200, Train Loss: 0.06901
Epoch: 164/200, Train Loss: 0.06722
Epoch: 165/200, Train Loss: 0.06974
Epoch: 166/200, Train Loss: 0.06998
Epoch: 167/200, Train Loss: 0.06545
Epoch: 168/200, Train Loss: 0.06892
Epoch: 169/200, Train Loss: 0.06737
Epoch: 170/200, Train Loss: 0.06958
Epoch: 171/200, Train Loss: 0.07307
Epoch: 172/200, Train Loss: 0.06520
Epoch: 173/200, Train Loss: 0.07175
Epoch: 174/200, Train Loss: 0.07272
Epoch: 175/200, Train Loss: 0.06651
Epoch: 176/200, Train Loss: 0.07164
Epoch: 177/200, Train Loss: 0.06569
Epoch: 178/200, Train Loss: 0.07343
Epoch: 179/200, Train Loss: 0.07257
Epoch: 180/200, Train Loss: 0.06926
Epoch: 181/200, Train Loss: 0.06531
Epoch: 182/200, Train Loss: 0.06998
Epoch: 183/200, Train Loss: 0.06389
Epoch: 184/200, Train Loss: 

Epoch: 188/200, Train Loss: 0.06953
Epoch: 189/200, Train Loss: 0.06919
Epoch: 190/200, Train Loss: 0.06820
Epoch: 191/200, Train Loss: 0.06905
Epoch: 192/200, Train Loss: 0.06968
Epoch: 193/200, Train Loss: 0.06920
Epoch: 194/200, Train Loss: 0.07241
Epoch: 195/200, Train Loss: 0.06775
Epoch: 196/200, Train Loss: 0.07140
Epoch: 197/200, Train Loss: 0.06775
Epoch: 198/200, Train Loss: 0.07241
Epoch: 199/200, Train Loss: 0.06824
Epoch: 200/200, Train Loss: 0.07000
Epoch: 1/200, Train Loss: 0.55905
Epoch: 2/200, Train Loss: 0.41700
Epoch: 3/200, Train Loss: 0.36574
Epoch: 4/200, Train Loss: 0.27080
Epoch: 5/200, Train Loss: 0.22615
Epoch: 6/200, Train Loss: 0.20086
Epoch: 7/200, Train Loss: 0.17594
Epoch: 8/200, Train Loss: 0.19743
Epoch: 9/200, Train Loss: 0.17001
Epoch: 10/200, Train Loss: 0.17125
Epoch: 11/200, Train Loss: 0.16761
Epoch: 12/200, Train Loss: 0.13467
Epoch: 13/200, Train Loss: 0.11801
Epoch: 14/200, Train Loss: 0.10414
Epoch: 15/200, Train Loss: 0.09798
Epoch: 16/200, T

Epoch: 20/200, Train Loss: 0.07789
Epoch: 21/200, Train Loss: 0.08745
Epoch: 22/200, Train Loss: 0.08221
Epoch: 23/200, Train Loss: 0.08051
Epoch: 24/200, Train Loss: 0.07416
Epoch: 25/200, Train Loss: 0.07410
Epoch: 26/200, Train Loss: 0.07797
Epoch: 27/200, Train Loss: 0.07706
Epoch: 28/200, Train Loss: 0.07188
Epoch: 29/200, Train Loss: 0.07366
Epoch: 30/200, Train Loss: 0.07412
Epoch: 31/200, Train Loss: 0.07426
Epoch: 32/200, Train Loss: 0.07091
Epoch: 33/200, Train Loss: 0.07527
Epoch: 34/200, Train Loss: 0.07415
Epoch: 35/200, Train Loss: 0.07338
Epoch: 36/200, Train Loss: 0.07312
Epoch: 37/200, Train Loss: 0.07053
Epoch: 38/200, Train Loss: 0.06955
Epoch: 39/200, Train Loss: 0.07017
Epoch: 40/200, Train Loss: 0.07120
Epoch: 41/200, Train Loss: 0.06836
Epoch: 42/200, Train Loss: 0.07173
Epoch: 43/200, Train Loss: 0.06803
Epoch: 44/200, Train Loss: 0.07496
Epoch: 45/200, Train Loss: 0.07014
Epoch: 46/200, Train Loss: 0.07218
Epoch: 47/200, Train Loss: 0.07327
Epoch: 48/200, Train

Epoch: 52/200, Train Loss: 0.06909
Epoch: 53/200, Train Loss: 0.06962
Epoch: 54/200, Train Loss: 0.07248
Epoch: 55/200, Train Loss: 0.06627
Epoch: 56/200, Train Loss: 0.06851
Epoch: 57/200, Train Loss: 0.06739
Epoch: 58/200, Train Loss: 0.06631
Epoch: 59/200, Train Loss: 0.06643
Epoch: 60/200, Train Loss: 0.07101
Epoch: 61/200, Train Loss: 0.06671
Epoch: 62/200, Train Loss: 0.06761
Epoch: 63/200, Train Loss: 0.06635
Epoch: 64/200, Train Loss: 0.06516
Epoch: 65/200, Train Loss: 0.06756
Epoch: 66/200, Train Loss: 0.06887
Epoch: 67/200, Train Loss: 0.06627
Epoch: 68/200, Train Loss: 0.06512
Epoch: 69/200, Train Loss: 0.06465
Epoch: 70/200, Train Loss: 0.06774
Epoch: 71/200, Train Loss: 0.06508
Epoch: 72/200, Train Loss: 0.06613
Epoch: 73/200, Train Loss: 0.06876
Epoch: 74/200, Train Loss: 0.06929
Epoch: 75/200, Train Loss: 0.06875
Epoch: 76/200, Train Loss: 0.06443
Epoch: 77/200, Train Loss: 0.06600
Epoch: 78/200, Train Loss: 0.06931
Epoch: 79/200, Train Loss: 0.06598
Epoch: 80/200, Train

Epoch: 84/200, Train Loss: 0.07069
Epoch: 85/200, Train Loss: 0.06766
Epoch: 86/200, Train Loss: 0.07700
Epoch: 87/200, Train Loss: 0.07223
Epoch: 88/200, Train Loss: 0.07064
Epoch: 89/200, Train Loss: 0.07197
Epoch: 90/200, Train Loss: 0.06722
Epoch: 91/200, Train Loss: 0.07298
Epoch: 92/200, Train Loss: 0.07133
Epoch: 93/200, Train Loss: 0.07176
Epoch: 94/200, Train Loss: 0.07008
Epoch: 95/200, Train Loss: 0.07262
Epoch: 96/200, Train Loss: 0.07419
Epoch: 97/200, Train Loss: 0.07028
Epoch: 98/200, Train Loss: 0.06920
Epoch: 99/200, Train Loss: 0.06930
Epoch: 100/200, Train Loss: 0.07103
Epoch: 101/200, Train Loss: 0.07431
Epoch: 102/200, Train Loss: 0.07548
Epoch: 103/200, Train Loss: 0.06822
Epoch: 104/200, Train Loss: 0.07264
Epoch: 105/200, Train Loss: 0.07279
Epoch: 106/200, Train Loss: 0.07206
Epoch: 107/200, Train Loss: 0.07042
Epoch: 108/200, Train Loss: 0.07009
Epoch: 109/200, Train Loss: 0.07176
Epoch: 110/200, Train Loss: 0.07112
Epoch: 111/200, Train Loss: 0.07148
Epoch: 1